In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Data Analysis**

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/ds/PlantVilage/Tomato",
    seed=123,
    shuffle=True,
    image_size = (256,256),
    batch_size = 32
    )

In [ ]:
classes = dataset.class_names
classes

In [ ]:
print('we have', len(dataset), 'batches')

In [ ]:
for image, label in dataset.take(1):
  print(image.shape, ':a batch of 32 elements of images of size (256, 256), and 3 for RGB channels')
  print(label.numpy(), ':a batch of 32 elements of class labels, each is one of 5 classes')

**Data Visualization**

In [ ]:
plt.figure(figsize=(15, 15))
for i in range(15):
  ax = plt.subplot(3, 5, i + 1)
  plt.imshow(image[i].numpy().astype("uint8"))
  plt.title(classes[label[i]])
  plt.axis("off")


**data Splitting**

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
print(len(train_ds), len(test_ds), len(val_ds))

# **Data Preprocessing**

In [ ]:
# normalize the image pixel value (0-1 by dividing by 256)
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(256,256),
  layers.experimental.preprocessing.Rescaling(1.0/255)
  ])

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

# **Model Creation**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
model = Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = (256,256)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='softmax')
])

model.build(input_shape = (32,256,256,3))

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
m = model.fit(
    train_ds,
    batch_size=32,
    validation_data=val_ds,
    epochs=30,
)

In [ ]:
score = model.evaluate(test_ds)
score

In [ ]:
accuracy = m.history['accuracy']
val_accuracy = m.history['val_accuracy']

loss = m.history['loss']
val_loss = m.history['val_loss']

In [ ]:
plt.figure(figsize = (10,10))
plt.subplot(1,2,1)
plt.plot(range(30), accuracy, label='Training Accuracy')
plt.plot(range(30), val_accuracy, label='Validation Accuracy')
plt.legend(loc='best')
plt.title('Training and Testing Accuracy')
plt.plot()

In [ ]:
plt.figure(figsize = (10,10))
plt.subplot(1,2,2)
plt.plot(range(30), loss, label='Training Loss')
plt.plot(range(30), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Testing Loss')
plt.plot()

# **Prediction**

In [ ]:
import numpy as np

# use only 1 batch of 32 images
for image, label in test_ds.take(1): 
  plt.imshow(image[0].numpy().astype('uint8'))
  print('Actual Label =' ,classes[label[0]])

  pred = model.predict(image)
  print('Predicted Label = ', classes[np.argmax(pred[0])])

In [ ]:
for image, label in test_ds.take(1): 
  plt.imshow(image[8].numpy().astype('uint8'))
  print('Actual Label =' ,classes[label[8]])

  pred = model.predict(image)
  print('Predicted Label = ', classes[np.argmax(pred[8])])

In [ ]:
for image, label in test_ds.take(1): 
  plt.imshow(image[11].numpy().astype('uint8'))
  print('Actual Label =' ,classes[label[11]])

  pred = model.predict(image)
  print('Predicted Label = ', classes[np.argmax(pred[11])])

Correct Predictions ✔✔✔

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0) # create a batch

    predictions = model.predict(img_array)

    predicted_class = classes[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = classes[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")